Импорт библиотек:

In [ ]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os # Для создания папок
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from scipy.stats import hmean
import random

Функция, переводящая unixtime в дату и время:
 - принимает значение в формате unixtime
 - возвращает две строки:
    - дату в формате день.месяц.год
    - время в формате часы:минуты

In [ ]:
def unixtime_to_date_time(unixtime):
    # Переводим unixtime в объект datetime (UTC)
    dt = datetime.datetime.utcfromtimestamp(unixtime)
    # Получаем форматы строки
    date = dt.strftime('%d.%m.%Y')
    time = dt.strftime('%H:%M')
    return date, time

Функция для расчета стабильного среднего количества сообщений в месяц и в неделю:
- принимает значения:
    - file (str) - путь до файла с таблицей в формате .xlsx
    - name (str) - имя листа, с которого будет обрабатываться информация
    - message_type (str) - тип сообщения (пост/комментарий)
- в результате выполнения выводит информацию о среднем количестве сообщений, за месяц и неделю, а также стандартное отклонение и коэффициент вариации относительно среднего количества за период (месяц/неделя)

In [ ]:
def stable(file='test.xlsx', name='Sheet1', message_type='пост'):
    df = pd.read_excel(file, sheet_name= name)
    df=df[df['message_type'] == message_type]
    df['parsed_date'] = pd.to_datetime(df['unixdate'], unit='s', errors='coerce')
    # Добавляем год и месяц
    df['year'] = df['parsed_date'].dt.year
    df['month'] = df['parsed_date'].dt.month
    # Группируем по месяцам и считаем посты
    monthly_counts = df.groupby(['year', 'month']).size().reset_index(name='post_count')
    # Считаем статистику
    mean_monthly = monthly_counts['post_count'].mean()  # Среднее в месяц
    std_monthly = monthly_counts['post_count'].std()    # Стандартное отклонение
    cv_monthly = 100 * std_monthly / mean_monthly if mean_monthly > 0 else 0  # Коэффициент вариации в %
    print(f"Стабильное (среднее) количество постов в месяц: {mean_monthly:.2f}")
    print(f"Стандартное отклонение: {std_monthly:.2f}")
    print(f"Коэффициент вариации: {cv_monthly:.1f}%")
    # То же для недели
    df['week'] = df['parsed_date'].dt.isocalendar().week
    weekly_counts = df.groupby(['year', 'week']).size().reset_index(name='post_count')
    mean_weekly = weekly_counts['post_count'].mean()
    std_weekly = weekly_counts['post_count'].std()
    cv_weekly = 100 * std_weekly / mean_weekly if mean_weekly > 0 else 0
    print(f"Стабильное (среднее) количество постов в неделю: {mean_weekly:.2f}")
    print(f"Стандартное отклонение по неделям: {std_weekly:.2f}")
    print(f"Коэффициент вариации по неделям: {cv_weekly:.1f}%")

Функция расчета средних значений (по остатку):
- принимает значения:
    - file (str) - путь до файла с таблицей в формате .xlsx
    - name (str) - имя листа, с которого будет обрабатываться информация
    - message_type (str) - тип сообщения (пост/комментарий)
    - target (str) - критерий расчета (имя столбца в таблице)
- возвращает 5 средний занчений по остатку

In [ ]:
def avg(file='test.xlsx', name='Sheet1', message_type='пост', target='views'):
    df = pd.read_excel(file, sheet_name=name)
    filter_column = 'message_type'
    #Отфильтровываем message_type по условию
    filtered_df = df[df[filter_column] == message_type]
    #Считаем первое среднее
    first_average = filtered_df[target].mean()
    #Отрезаем верх
    filtered_df_for_second_avg = filtered_df[filtered_df[target] < first_average]
    #Второе среднее
    second_average = filtered_df_for_second_avg[target].mean()
    #и так далее
    filtered_df_for_third_avg = filtered_df_for_second_avg[filtered_df_for_second_avg[target] < second_average]
    third_average = filtered_df_for_third_avg[target].mean()
    filtered_df_for_fourth_avg = filtered_df_for_third_avg[filtered_df_for_third_avg[target] < third_average]
    fourth_average = filtered_df_for_fourth_avg[target].mean()
    filtered_df_for_fifth_avg = filtered_df_for_fourth_avg[filtered_df_for_fourth_avg[target] < fourth_average]
    fifth_average = filtered_df_for_fifth_avg[target].mean()
    return(first_average, second_average, third_average, fourth_average, fifth_average)

Функция построения графика по месяцам + среднее значение:
- принимает значения:
    - file_input (str) - путь до файла формата .xlsx
    - file_output (str) - путь и имя созраняемого графика
    - type (str) - тип сообщения (пост/комментарий)
    - avg_mark (str) - характеристика, по которой строится график (имя столбца)
    - name (str) - имя листа, с которого будет обрабатываться информация
- в результате выполнения сохраняет график с значениями характеристики за месяц в указанный файл

In [ ]:
def graph_avg_month(file_input='representative.xlsx', file_output='graph.png', type='пост', avg_mark='sn', name='Sheet1'):
    df=pd.read_excel(file_input, sheet_name=name)
    df['parsed_date'] = pd.to_datetime(df['unixdate'], unit='s', errors='coerce')
    df_posts = df[df['message_type'].str.contains(type, case=False, na=False)].copy()
    # 2. Извлекаем год и месяц как отдельные столбцы
    df_posts['year_month'] = df_posts['parsed_date'].dt.to_period('M')
    # 3. Группируем по месяцам, суммируя лайки и просмотры
    monthly = df_posts.groupby('year_month').agg({avg_mark:'sum'}).reset_index()
    avg=monthly[avg_mark].mean()
    # 4. График с двумя осями
    fig, ax1 = plt.subplots(figsize=(13,6))
    color = 'tab:blue'
    ax1.set_xlabel('Месяц')
    ax1.set_ylabel(avg_mark, color=color)
    ax1.plot(monthly['year_month'].astype(str), monthly[avg_mark], color=color, marker='o', label='Количество ')
    plt.axhline(y=avg, color='g', linestyle='--', linewidth=1, label=f'Среднее: {avg:.2f}')
    ax1.tick_params(axis='y', labelcolor=color)
    plt.xticks(rotation=45)
    plt.title('Количество по месяцам')
    plt.grid(True)
    fig.tight_layout()
    plt.savefig(file_output)
    plt.show()
    plt.close()

Функция для построения графика поступления сообщений по времени суток:
- принимает значения:
    - file_input (str) - путь до файла формата .xlsx
    - file_output (str) - путь и имя созраняемого графика
    - type (str) - тип сообщения (пост/комментарий)
    - avg_mark (str) - характеристика, по которой строится график (имя столбца)
    - name (str) - имя листа, с которого будет обрабатываться информация
- в результате выполнения сохраняет график времени появления сообщений в указанный файл

In [ ]:
def graph_avg_time(file_input='representative.xlsx', file_output='graph.png', type='пост', avg_mark='sn', name='Sheet1'):
    df=pd.read_excel(file_input, sheet_name=name)
    df['parsed_time'] = pd.to_datetime(df['unixdate'], unit='s', errors='coerce')
    df_posts = df[df['message_type'].str.contains(type, case=False, na=False)].copy()
    df_posts['hour_of_day'] = df_posts['parsed_time'].dt.hour
    monthly = df_posts.groupby('hour_of_day').agg({avg_mark:'sum'}).reset_index()
    avg=monthly[avg_mark].mean()
    # 4. График с двумя осями
    fig, ax1 = plt.subplots(figsize=(11,6))
    color = 'tab:blue'
    ax1.set_xlabel('Час')
    ax1.set_ylabel(avg_mark, color=color)
    ax1.plot(monthly['hour_of_day'].astype(str), monthly[avg_mark], color=color, marker='o', label='Количество ')
    plt.axhline(y=avg, color='g', linestyle='--', linewidth=1, label=f'Среднее: {avg:.2f}')
    ax1.tick_params(axis='y', labelcolor=color)
    plt.xticks(rotation=45)
    plt.title('Количество по времени')
    plt.grid(True)
    fig.tight_layout()
    plt.savefig(file_output)
    plt.show()
    plt.close()

Функция, оценивающая посты с разным количеством [просмотров/лайков/комментариев] в выборке (разбиение на группы):
- принимает значения:
    - df (data_frame) - структура данных pandas, содержащая данные из .xlsx таблицы
    - column (str) - имя столбца таблицы, по которому производится оценка
- возвращает количество записей в каждом диапазоне в порядке меток

In [ ]:
def count_views_groups(df, column='views'):
    # Приводим к числу, если вдруг в данных текст или пропуски
    df[column] = pd.to_numeric(df[column], errors='coerce')
    df = df[df['message_type'].str.contains('пост', case=False, na=False)].copy()
    
    bins = [float('-inf'), 100, 500, 1000, 5000, 10000, 100000, float('inf')]
    labels = [
        'менее 100',
        'от 100 до 500',
        'от 500 до 1000',
        'от 1000 до 5000',
        'от 5000 до 10000',
        'от 10000 до 100000',
        'более 100000'
    ]
    
    # Разбивка на интервалы
    df['views_range'] = pd.cut(df[column], bins=bins, labels=labels, right=False)
    
    # Подсчет количества в каждой группе
    return df['views_range'].value_counts(sort=False)


Функция для сбора имен и дат регистрации пользователей соц. сети ВКонтакте:
- принимает ссылку/id профиля пользователя соц. сети ВКонтакте
- возвращает два строковых значения:
    - имя пользователя
    - дату регистрации в соц. сети ВКонтакте

In [ ]:
def get_vk_info_selenium(vk_link_or_id):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    try:
        driver.get('https://regvk.com/')

        input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'enter'))
        )
        input_field.send_keys(vk_link_or_id)

        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@type="submit"]'))
        )
        submit_button.click()

        # Дожидаемся загрузки блока с результатом
        result_block = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, 'result'))
        )

        # 1. Извлекаем имя пользователя
        # Имя находится в h3 внутри div с классом 'info' внутри result_block
        try:
            name_element = result_block.find_element(By.XPATH, './/div[@class="info"]/table/tbody/tr/td/h3')
            user_name = name_element.text.strip()
        except:
            user_name = "Имя пользователя не найдено" # Если h3 не найден

        # 2. Извлекаем дату регистрации
        try:
            reg_date_element = result_block.find_element(By.XPATH, './/td[contains(text(), "Дата регистрации:")]')
            reg_date_text = reg_date_element.text.replace('Дата регистрации:', '').replace(' года', '').strip() # Убираем " года"
        except:
            reg_date_text = "Дата регистрации не найдена"

        # 3. Проверяем наличие ошибки "Страница не найдена."
        error_message_elements = driver.find_elements(By.XPATH, '//*[contains(text(), "Страница не найдена.")]')
        if error_message_elements:
            return {'error': "Страница не найдена."}
        
        return {
            'name': user_name,
            'reg_date_text': reg_date_text
        }

    except Exception as e:
        print(f"Произошла ошибка при получении данных: {e}")
        return {'error': f"Не удалось получить данные или найти элементы: {e}"}
    finally:
        driver.quit() # Всегда закрывать браузер

Вспомогательная функция для форматирования даты:
- принимает дату в формате строки [число] [название месяца] [год]
- возвращает дату - строку в формате день.номер_месяца.год

In [ ]:
def format_date(date_str):

    if not date_str or date_str == "Дата регистрации не найдена":
        return None

    month_mapping = {
        'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4,
        'мая': 5, 'июня': 6, 'июля': 7, 'августа': 8,
        'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12
    }

    try:
        parts = date_str.split()
        day = int(parts[0])
        month_name = parts[1].lower()
        year = int(parts[2])

        month = month_mapping[month_name]
        # Здесь datetime – это модуль, поэтому обращаемся к классу явно:
        date_obj = datetime.datetime(year, month, day)
        return date_obj.strftime('%d.%m.%Y')
    except Exception as e:
        print(f"Ошибка форматирования даты '{date_str}': {e}")
        return None

Функция, формирующая выборку из 1000 случайных задписей:
- принимает значения:
    - df (data_frame) - структура данных pandas, содержащая данные из .xlsx таблицы
    - sample_size (int) - размер случайной выборки (количество строк)
- возвращает структуру данных pandas с заданным количеством случайных строк из изначальной структуры

In [ ]:
def get_random_sample(df, sample_size=1000):
    random_sample_df = df.sample(n=sample_size, random_state=42)
    return random_sample_df

Функция для подсчета сообщений, содержащих ключевые слова из словаря:
- принимает занчения:
    - df (data_frame) - структура данных pandas, содержащая данные из .xlsx таблицы
    - text_column (str) - имя столбца таблицы, по которому производится поиск
    - keywords_dict (dic) - словарь, содержащий различные группы ключевых слов
- возвращает количество вхождений слов любого из списков, общее количество входжений слов из словаря и среднее количество просмотров на этих постах в виде словаря.

In [ ]:
def count_posts_with_keywords(df, text_column, keywords_dict):
    results = {}
    
    # Убедимся, что столбец с текстом существует и является строковым
    if text_column not in df.columns:
        print(f"Ошибка: Столбец '{text_column}' не найден в DataFrame.")
        return results
    
    # Заменяем NaN в текстовом столбце на пустые строки, чтобы str.contains не выдавал ошибку
    df[text_column] = df[text_column].fillna('')
    all_keywords_flat = [] # Список всех ключевых слов для общего подсчета
    for category, keywords in keywords_dict.items():
        if not keywords: # Пропускаем пустые списки ключевых слов
            results[category] = 0
            continue
        # Создаем регулярное выражение для текущей категории
        pattern = r'\b(?:' + '|'.join(re.escape(word) for word in keywords) + r')\b'
        
        # Подсчитываем посты, содержащие любое из слов в текущей категории
        contains_keywords = df[text_column].str.contains(pattern, case=False, regex=True)
        
        results[category] = contains_keywords.sum()
        all_keywords_flat.extend(keywords) # Добавляем для общего подсчета
    # Подсчитываем общее количество постов, содержащих хотя бы одно слово из любого списка
    if all_keywords_flat:
        overall_pattern = r'\b(?:' + '|'.join(re.escape(word) for word in set(all_keywords_flat)) + r')\b'
        # 4. Собираем в overall данные по просмотрам таких постов
        overall_contains_keywords = df[text_column].str.contains(overall_pattern, case=False, regex=True)
        overall = df.loc[overall_contains_keywords, 'views']
        #overall = overall.tolist()
        results['Общее количество постов с любым из ключевых слов'] = overall_contains_keywords.sum()
        results['Среднее количество просмотров на посте']=overall.mean()
    else:
        results['Общее количество постов с любым из ключевых слов'] = 0
    return results

Оценка постов с наибольшим количеством комментариев:

In [ ]:
# Загрузка данных
data = pd.read_excel('data.xlsx')
comments_table = pd.read_excel('random_1000comments.xlsx')
# 1. Анализ данных из таблицы data (посты)
posts = data[(data['message_type'] == 'пост') & (data['comments'] > 0)]
posts = posts[['message_url', 'text', 'comments']].copy()
# Создаем столбцы для подсчета ботов и людей
posts['bot_count'] = 0
posts['hum_count'] = 0
# 2. Анализ комментариев и определение ботов/людей
for index, post in posts.iterrows():
    post_url = post['message_url']
    
    # Выбираем комментарии к текущему посту
    post_comments = data[(data['message_type'] == 'комментарий') & (data['message_url'].str.startswith(post_url))]
    
    bot_count = 0
    hum_count = 0
    
    for comment_index, comment in post_comments.iterrows():
        comment_from_id = comment['from_id']
        vk_id = 'https://vk.com/id' + str(comment_from_id)
        
        # Ищем соответствие в таблице comments
        match = comments_table[comments_table['from_id'] == vk_id]
        
        if not match.empty:
            bot_mark = match['bot_mark'].values[0]
            if bot_mark != 0:
                bot_count += 1
            else:
                hum_count += 1
    
    # Обновляем значения в posts
    posts.loc[index, 'bot_count'] = bot_count
    posts.loc[index, 'hum_count'] = hum_count
# Сохранение результата
posts.to_excel('hipothetic.xlsx', index=False)